Script per provare la classe definita per l'analisi e la valutazione dei diversi keygenes.   


# Packages

In [1]:
import pandas as pd 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Nasconde i FutureWarning

# Per aggiungere un path per dei pacchetti selfmade
import sys
sys.path.append('/Users/mattia/Desktop/Università/Dottorato/Python/EvaluateKeyGenes/__HandmadePackages__')

# Classe per agevolare la valutazione della performance di un subset di keygenes
from EvaluatePipeline import Evaluate

# Funzioni selfmade per preprocessing
import preprocessing

# Classe selfmade per creare report LaTeX
from LaTeX import LaTeX, knit

# Manage the data

## Import the data

In [2]:
#df_train = pd.read_csv("/Users/mattia/Desktop/Università/Magistrale/Tesi/Python TMM only for exploration/Data/GBM_DEGS_normalized_counts_train.csv")
#df_train = pd.read_csv("/Users/mattia/Desktop/Università/Magistrale/Tesi/Python TMM only for exploration/Data/GBM_DEGREE_normalized_counts_train.csv")
df_train = pd.read_csv("/Users/mattia/Desktop/Università/Dottorato/TEPs HC vs GBM/Train_GSE68086_Test_GSE156902/Data Product/normalizedcounts_trained_onGSE68086_GBM_HC_BETWEENNESS.csv")
#df_train = pd.read_csv("/Users/mattia/Desktop/Università/Magistrale/Tesi/Python TMM only for exploration/Data/GBM_CLOSENESS_normalized_counts_train.csv")
#df_train = pd.read_csv("/Users/mattia/Desktop/Università/Magistrale/Tesi/Python TMM only for exploration/Data/GBM_CLOSENESS_last5_normalized_counts_train.csv")
#df_train = pd.read_csv("/Users/mattia/Desktop/Università/Magistrale/Tesi/Python TMM only for exploration/Data/GBM_EIGEN_normalized_counts_train.csv")
#df_train = pd.read_csv("/Users/mattia/Desktop/Università/Magistrale/Tesi/Python TMM only for exploration/Data/GBM_CLUSTER_normalized_counts_train.csv")

print(f"df_train shape: {df_train.shape}")

df_test1 = pd.read_csv("/Users/mattia/Desktop/Università/Dottorato/TEPs HC vs GBM/Train_GSE68086_Test_GSE156902/Data Product/testdata_normalized_GSE156902_GBM_HC.csv")
print(f"df_test1 shape: {df_test1.shape}")

df_train shape: (95, 44)
df_test1 shape: (322, 4489)


In [3]:
pd.value_counts(df_test1["condition"])

condition
HC     252
GBM     70
Name: count, dtype: int64

## Visualize data

In [4]:
display(df_train.head())
display(df_test1.head())

,Unnamed: 0,ENSG00000012822,ENSG00000013016,ENSG00000034152,ENSG00000054267,ENSG00000069966,ENSG00000088888,ENSG00000090273,ENSG00000093167,ENSG00000096060,...,ENSG00000162191,ENSG00000174695,ENSG00000177556,ENSG00000178741,ENSG00000180098,ENSG00000182054,ENSG00000183255,ENSG00000185515,ENSG00000189343,condition
0,Control-10,7.038585,5.807677,6.013357,6.689549,8.324849,6.113623,6.106153,6.767386,-0.781046,...,4.831676,4.485204,3.996339,3.304507,1.244903,5.567719,7.022802,2.882967,5.712495,HC
1,Control-13,6.655492,4.772204,5.878212,7.290456,7.579616,5.297315,5.756760,4.345366,4.345366,...,8.033556,4.604468,3.533503,5.984545,4.496787,5.736612,6.314053,3.984459,7.936802,HC
2,Control-2,7.063475,5.683803,6.112908,5.813173,8.364684,5.434382,4.988357,6.231613,-0.069593,...,5.086128,4.441404,4.110635,5.005119,2.784149,5.923098,6.509328,3.800806,5.704477,HC
3,Control-3,5.609102,4.669565,6.071938,5.169899,8.046189,6.047491,4.307266,5.695553,-0.781046,...,6.336931,4.535356,4.002699,4.101003,4.711663,5.757152,6.734132,3.860252,5.139379,HC
4,Control-4,6.356867,4.963658,5.609464,5.795295,7.915064,5.438589,5.681984,4.924339,0.002054,...,5.162624,5.057510,3.807902,4.485099,4.430014,4.755378,6.356867,3.971150,6.257535,HC


,Unnamed: 0,ENSG00000000419,ENSG00000000938,ENSG00000001036,ENSG00000001461,ENSG00000001629,ENSG00000001631,ENSG00000002330,ENSG00000002549,ENSG00000002586,...,ENSG00000267316,ENSG00000268555,ENSG00000269028,ENSG00000270055,ENSG00000271043,ENSG00000272053,ENSG00000272168,ENSG00000272369,ENSG00000272888,condition
0,AMC-HD-001-TR1139,5.842468,5.935578,5.023041,2.257506,2.520540,5.145031,3.935578,6.001667,10.927100,...,7.327895,5.064861,8.466959,4.183505,6.818221,10.677887,4.395010,4.742933,4.579434,HC
1,AMC-HD-002-TR1140,3.012584,5.366221,3.749549,3.543098,3.749549,3.650014,3.930122,3.650014,10.936736,...,7.164587,5.725302,8.895227,-0.157341,7.420088,8.881578,5.012584,5.427621,4.366221,HC
2,AMC-HD-003-TR1141,4.616159,4.698621,5.229136,3.336051,3.698621,2.113658,4.850624,3.528696,11.508836,...,7.323112,4.336051,9.892831,3.850624,8.471210,9.012512,5.573090,5.698621,4.435587,HC
3,AMC-HD-004-TR1142,1.241248,4.048603,3.700680,0.241248,1.241248,3.241248,4.885104,2.241248,11.815369,...,7.086738,2.241248,9.038910,4.241248,7.563176,8.117765,5.285642,3.563176,4.885104,HC
4,AMC-HD-005-TR1143,3.462869,5.150925,4.100298,3.752375,3.993383,4.462869,5.100298,4.816505,11.448394,...,7.432495,5.380406,9.968901,2.292944,8.264487,8.922300,4.614872,4.752375,4.380406,HC


## Cleaning data

In [5]:
# Cleaning train dataset 
df_train.rename(columns={"Unnamed: 0":"patient"},inplace=True)
df_train.set_index("patient",inplace=True)
cleaning_output_df_train = preprocessing.feature_to_numeric(dataset=df_train,
                                                            feature_name="condition",
                                                            verbose=False)
df_train = cleaning_output_df_train["dataset"].copy()

#----#
# Cleaning test1 dataset 
df_test1.rename(columns={"Unnamed: 0":"patient"},inplace=True)
df_test1.set_index("patient",inplace=True)
cleaning_output_df_test1 = preprocessing.feature_to_numeric(dataset=df_test1,
                                                            feature_name="condition",
                                                            verbose=False)
df_test1 = cleaning_output_df_test1["dataset"].copy()

Visualizzare la ricodifica della variabile **cancer**, capire, essendo binaria, a cosa è stato assegnato **0** ed a cosa è stato assegnato **1**. 


In [6]:
cleaning_output_df_train["labels"]["condition"]

{0: 'GBM', 1: 'HC'}

In [7]:
cleaning_output_df_test1["labels"]["condition"]

{0: 'GBM', 1: 'HC'}

# Try the class

In [8]:
gbm_subset1_evaluation = Evaluate(train_data=df_train,
                                  test_data=df_test1,
                                  seed=0,
                                  target_feature_name="condition",
                                  labels = cleaning_output_df_test1["labels"]["condition"],
                                  verbose=True
                                 )
gbm_subset1_evaluation.LogisticRegression(verbose=False)
gbm_subset1_evaluation.SupportVectorClassifier(verbose=False)
gbm_subset1_evaluation.SupportVectorClassifierKFold(metric_to_optimize="f1",
                                                    class_to_optimize=0,
                                                    stratify=True,
                                                    fold=5,
                                                    param_grid= { 'C': [0.01,0.05,0.1, 1, 10, 100],   
                                                                  'kernel': ['linear', 'rbf'],
                                                                  'gamma': ['scale', 'auto']},
                                                    verbose=False
                                                   )

gbm_subset1_evaluation.show_performance(modelli=["SVC"],verbose=True) # non funziona il suggeritore
risultati = gbm_subset1_evaluation.show_performance(modelli=["SVC","SVC_KF","LR"],verbose=False) # non funziona il suggeritore

train_data shape: (95, 43)
test_data shape: (322, 4488)
Numero di geni comuni: 37
Seed utilizzato: 0
Labels utilizzate: {'0': 'GBM', '1': 'HC'}



Support Vector Classifier


,GBM,HC,accuracy,macro avg,weighted avg
precision,0.830769,0.937743,0.916149,0.884256,0.914488
recall,0.771429,0.956349,0.916149,0.863889,0.916149
f1-score,0.800000,0.946955,0.916149,0.873477,0.915008
support,70.000000,252.000000,0.916149,322.000000,322.000000


,Accuracy,Precision,Recall,F1-Score,Specificity
GBM,0.916149,0.830769,0.771429,0.800000,0.956349
HC,0.916149,0.937743,0.956349,0.946955,0.771429
